In [1]:
from decomp_utils import *
import tensorflow as tf
import numpy as np
import tensorflow_addons as tfa
from sklearn.metrics import confusion_matrix
import tensorflow_probability as tfp

D:\anaconda\envs\decomp\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.7.0 and strictly below 2.10.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.10.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [2]:
path = "D:\\AI-Workspace\\Decomposition\\ViT_Decompose\\data\\5_50_GM-"
train_ls = [1, 3]
test_ls = 2
overlap = 5
x_sEMG = 'EMGs'
y_spikes = 'Spikes'
mu = [0,1,2,3,4,5,6,7,8,9]

In [3]:
x, y = load_train(path, train_ls, overlap,mu)
x = np.expand_dims(x, axis=3)  

In [4]:
input_shape = x.shape[1:]
output_shape = 1

In [5]:
#Hyper-parameter

input_shape = input_shape
patch_size = 12
no_patches = 50  # h * w / p^2
dims = 100
no_transformer_layers = 4
no_heads = 2
transformer_units = [dims * 2,dims]  # Size of the transformer layers
mlp_head_units = [240, 10]   # [2048, 1024]
num_classes = 1
learning_rate = 0.001
weight_decay = 0.0001

In [6]:
class Generate_Patches(tf.keras.layers.Layer):
    
    def __init__(self, patch_size):
        super(Generate_Patches, self).__init__()
        self.patch_size = patch_size
    
    
    def call(self, images):
        
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(images=images,
                           sizes=[1, self.patch_size, self.patch_size, 1],
                           strides=[1, self.patch_size, self.patch_size, 1],
                           rates=[1, 1, 1, 1],
                           padding='VALID')
        h_w_c = patches.shape[-1]
        patch_reshape = tf.reshape(patches, [batch_size, -1, h_w_c]) # (batch_size, no_of_patches, h*w*c)
        
        return patch_reshape

In [7]:
class Embed_Position(tf.keras.layers.Layer):
    
    def __init__(self, num_patches, projection_dims):
        
        super(Embed_Position, self).__init__()
        
        self.num_patches = num_patches
        self.project = tf.keras.layers.Dense(units=projection_dims)
        self.embed = tf.keras.layers.Embedding(input_dim=num_patches, output_dim=projection_dims)
    
    def call(self, patch):
        
        position = tf.range(start=0, limit=self.num_patches, delta=1)
        encode = self.project(patch) + self.embed(position)
        
        return encode

In [8]:
def mlp_mixer(inputs, no_units, drop_out):
    
    for units in no_units:
        x = tf.keras.layers.Dense(units=units, activation = tf.nn.gelu)(inputs)
        x = tf.keras.layers.Dropout(drop_out)(x)
        # x = tf.keras.layers.Dense(inputs.shape[-1], activation=tf.nn.gelu)(x)
    
    return x

In [9]:
def mo_vit_trans_decomp():
    
    outputs = []
    no_of_nodes = len(mu)
    
    inputs = tf.keras.Input(shape=input_shape)
    #x = tf.keras.layers.BatchNormalization()(inputs)
    returned_patches = Generate_Patches(patch_size)(inputs)
    encoded_patches = Embed_Position(no_patches, dims)(returned_patches)
    #encoded_patches = tf.keras.layers.BatchNormalization()(encoded_patches)
    # Create multiple layers of the Transformer block.
    for _ in range(no_transformer_layers):
        # Layer normalization 1.
        input_two_attention = tf.keras.layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        # Create a multi-head attention layer.
        attention_output = tf.keras.layers.MultiHeadAttention(num_heads=no_heads, key_dim=dims, dropout=0.1)(input_two_attention, input_two_attention)
        # Skip connection 1.
        input_two_attention_2 = tf.keras.layers.Add()([attention_output, encoded_patches])
        # Layer normalization 2.
        input_two_attention_3  = tf.keras.layers.LayerNormalization(epsilon=1e-6)(input_two_attention_2)
        # MLP.
        input_two_attention_3 = mlp_mixer(input_two_attention_3, no_units=transformer_units, drop_out=0.1)
        # Skip connection 2.
        encoded_patches = tf.keras.layers.Add()([input_two_attention_3, input_two_attention_3])
        
        
# Create a [batch_size, projection_dim] tensor.
    representation = tf.keras.layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = tf.keras.layers.Flatten()(representation)
    representation = tf.keras.layers.Dropout(0.5)(representation)
    # Add MLP.
    features = mlp_mixer(representation, no_units=mlp_head_units, drop_out=0.5)
    # Classify outputs.
    
    for i in range(1, no_of_nodes+1):
        output = tf.keras.layers.Dense(output_shape, activation='sigmoid', name='output_{}'.format(i))(features)
        outputs.append(output)

    # Create the Keras model.
    model = tf.keras.Model(inputs=inputs, outputs=outputs, name="sEMG-Decomposition")
    
    return model

In [10]:
model = mo_vit_trans_decomp()
model.summary()

Model: "sEMG-Decomposition"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 120, 64, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 generate__patches (Generate_Pa  (None, None, 144)   0           ['input_1[0][0]']                
 tches)                                                                                           
                                                                                                  
 embed__position (Embed_Positio  (None, 50, 100)     19500       ['generate__patches[0][0]']      
 n)                                                                              

 dropout_7 (Dropout)            (None, 50, 100)      0           ['dense_8[0][0]']                
                                                                                                  
 add_7 (Add)                    (None, 50, 100)      0           ['dropout_7[0][0]',              
                                                                  'dropout_7[0][0]']              
                                                                                                  
 layer_normalization_8 (LayerNo  (None, 50, 100)     200         ['add_7[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 flatten (Flatten)              (None, 5000)         0           ['layer_normalization_8[0][0]']  
                                                                                                  
 dropout_8

In [11]:
accuracy_callback = AccuracyCallback('accuracy')
f1_callback = AccuracyCallback('f1_m')


n_batch = 128
n_epochs = 1
ls =  'binary_crossentropy'
mtr = ['mse', 'accuracy',  f1_m]
opt = 'rmsprop'
model.compile(optimizer=opt, loss=ls, metrics=mtr)

In [12]:
history = model.fit(x,y, validation_split=0.2, batch_size=n_batch, epochs=n_epochs)

129/129 [==============================] - 26s 116ms/step - loss: 4.2934 - output_1_loss: 1.2329 - output_2_loss: 0.3618 - output_3_loss: 0.3811 - output_4_loss: 0.3329 - output_5_loss: 0.2784 - output_6_loss: 0.5403 - output_7_loss: 0.3051 - output_8_loss: 0.3234 - output_9_loss: 0.2874 - output_10_loss: 0.2500 - output_1_mse: 0.4403 - output_1_accuracy: 0.3336 - output_1_f1_m: 0.0572 - output_2_mse: 0.1087 - output_2_accuracy: 0.9430 - output_2_f1_m: 0.0090 - output_3_mse: 0.1177 - output_3_accuracy: 0.9333 - output_3_f1_m: 0.0149 - output_4_mse: 0.0942 - output_4_accuracy: 0.9469 - output_4_f1_m: 0.0108 - output_5_mse: 0.0750 - output_5_accuracy: 0.9370 - output_5_f1_m: 0.0176 - output_6_mse: 0.1698 - output_6_accuracy: 0.8280 - output_6_f1_m: 0.0431 - output_7_mse: 0.0836 - output_7_accuracy: 0.9327 - output_7_f1_m: 0.0195 - output_8_mse: 0.0947 - output_8_accuracy: 0.9589 - output_8_f1_m: 0.0076 - output_9_mse: 0.0762 - output_9_accuracy: 0.9347 - output_9_f1_m: 0.0144 - output_10

In [13]:
# Test Validation

x_ts, y_ts = load_test(path, test_ls, overlap,mu)
x_ts = np.expand_dims(x_ts, axis=3)

In [14]:
model.evaluate(x_ts, y_ts)

319/319 [==============================] - 10s 32ms/step - loss: 1.9633 - output_1_loss: 0.7328 - output_2_loss: 0.1741 - output_3_loss: 0.1823 - output_4_loss: 0.1294 - output_5_loss: 0.0963 - output_6_loss: 0.2108 - output_7_loss: 0.1316 - output_8_loss: 0.1452 - output_9_loss: 0.1004 - output_10_loss: 0.0603 - output_1_mse: 0.2698 - output_1_accuracy: 0.0282 - output_1_f1_m: 0.0540 - output_2_mse: 0.0344 - output_2_accuracy: 0.9703 - output_2_f1_m: 0.0000e+00 - output_3_mse: 0.0349 - output_3_accuracy: 0.9734 - output_3_f1_m: 0.0000e+00 - output_4_mse: 0.0261 - output_4_accuracy: 0.9739 - output_4_f1_m: 0.0000e+00 - output_5_mse: 0.0192 - output_5_accuracy: 0.9804 - output_5_f1_m: 0.0000e+00 - output_6_mse: 0.0417 - output_6_accuracy: 0.9733 - output_6_f1_m: 0.0000e+00 - output_7_mse: 0.0280 - output_7_accuracy: 0.9712 - output_7_f1_m: 0.0000e+00 - output_8_mse: 0.0237 - output_8_accuracy: 0.9840 - output_8_f1_m: 0.0000e+00 - output_9_mse: 0.0202 - output_9_accuracy: 0.9794 - output

[1.9632724523544312,
 0.7328438758850098,
 0.1740976721048355,
 0.18227101862430573,
 0.12944242358207703,
 0.09632701426744461,
 0.21081386506557465,
 0.13159900903701782,
 0.14520803093910217,
 0.10036522895097733,
 0.060304976999759674,
 0.2698369324207306,
 0.02817869372665882,
 0.054045870900154114,
 0.03442078456282616,
 0.9702503681182861,
 0.0,
 0.0349278561770916,
 0.9733922481536865,
 0.0,
 0.026127323508262634,
 0.9738831520080566,
 0.0,
 0.019240327179431915,
 0.9803632497787476,
 0.0,
 0.04170413687825203,
 0.9732940793037415,
 0.0,
 0.028017982840538025,
 0.9712321758270264,
 0.0,
 0.02366774156689644,
 0.9839960932731628,
 0.0,
 0.02019151858985424,
 0.9793814420700073,
 0.0,
 0.008581788279116154,
 0.9919489622116089,
 0.0]

In [15]:
select_motor = 9

In [16]:
pred = model.predict(x_ts)
y_pred = np.argmax(pred[select_motor], axis=-1)
confusion = confusion_matrix(y_ts[select_motor], y_pred)

319/319 [==============================] - 4s 11ms/step


In [17]:
TP = confusion[0, 0]
TN = confusion[1, 1]
FP = confusion[1, 0]
FN = confusion[0, 1]

In [18]:
precision = TP / float(TP + FP)
print("Precision is {}".format(precision))

Precision is 0.9919489445262641


In [19]:
recall = TP / float(TP + FN)
print("recall is {}".format(recall))

#RECALL SAME AS SENSITIVITY

recall is 1.0


In [20]:
model.save('decompose')

INFO:tensorflow:Assets written to: decompose\assets


INFO:tensorflow:Assets written to: decompose\assets


In [21]:
from tensorflow.keras.utils import plot_model